In [2]:
import os 
import sys
import json
sys.path.append(os.path.abspath(".."))
from velopix_wrappers.optimizers import BaseOptimizer
from velopix_wrappers.velopix_pipeline import TrackFollowingPipeline, GraphDFSPipeline, SearchByTripletTriePipeline
from typing import Any, Dict
import numpy as np

from typing import Any, Dict, Literal, List
import random
import numpy as np
from velopix_wrappers.optimizers import BaseOptimizer, pMap
from copy import deepcopy

In [ ]:
class Node(object):
    def __init__(self, bounds, parent: Any = None):
        self.sum_reward = 0
        self.visited = 0
        self.children = []
        self.parent = parent
        self.bounds = bounds

    def add_child(self, child: Any):
        self.children.append(child)

In [ ]:
#return pmap based on bounds (split in middle) (rollout phase)
def returnPmap (bounds: Dict[str, Any], cfg: Dict[str, Any]) -> Dict[str, Any]:
    new_pmap = {}
    
    for key, (typ, _) in cfg.items():
        if typ is float:
            new_pmap[key] = (bounds[key][0] + bounds[key][1]) / 2
        elif typ is int:
            new_pmap[key] = (int)((bounds[key][0] + bounds[key][1]) / 2)
        elif typ is bool:
            new_pmap[key] = bounds[key]
        
    return new_pmap

In [ ]:
class PolyHoot(BaseOptimizer):
    def __init__(
        self,
        max_iterations: int = 100,
        objective: Literal["min", "max"] = "min"
    ):
        super().__init__(objective=objective)
        self.max_iterations = max_iterations
        self.current_iteration = 0

        self.alfa = 5
        self.epsilon = 20
        self.eta = 0.5


        cfg = self._algorithm.get_config()
        self.bounds = self._algorithm.get_bounds()
        self.root = Node(bounds=self.bounds)  # Root node with no bounds
        self.nodes = [self.root]

        self.param_num = 0

        for key, (typ, _) in cfg.items():
            if typ is not bool:
                self.param_num += 1

        self.nu = 4 * self.param_num
        self.ro = 1 / (4 * self.param_num)


    def init(self) -> pMap:
        """
        Initializes with a random point within bounds.
        """
        self.current_iteration += 1

        #pregenerate trees of bounds
        for key, (typ, _) in self.cfg.items():
            if typ is bool:
                for node in self.nodes:
                    if len(node.children) == 0:
                        bounds = node.bounds[key] = 0
                        node1 = Node(bounds=bounds, parent=node)
                        bounds = node.bounds[key] = 1
                        node2 = Node(bounds=bounds, parent=node)

                        self.nodes.append(node1)
                        self.nodes.append(node2)



        #sample node

        return initial_config
    


    def next(self) -> pMap:
        """
        Evaluates the current configuration and returns a new one.
        """
        self.current_iteration += 1

        # Evaluate the current configuration (from previous init/next call)
        score = self.objective_func([1.0, 1.0, 1.0, -10.0])
        
        # Record evaluation
        if self._current_config:
            # Update best if better
            if self.objective == "min":
                if score < self.best_score:
                    self.best_score = score
                    self.best_config = self._current_config.copy()
            else:  # "max"
                if score > self.best_score:
                    self.best_score = score
                    self.best_config = self._current_config.copy()
        
        
        #TODO: get node

        #TODO: split node


        pmap = returnPmap(node.bounds, self.cfg)

            
        return pmap
    

    def is_finished(self) -> bool:
        """Determines if optimization is complete."""
        #TODO: possibly add a check for reaching target score
        return self.current_iteration >= self.max_iterations
            

In [ ]:
events = []
n_files = 150

for i in range(0, n_files):
    if i == 51:
        """
        There's an issue with event 51 -> module_prefix_sum contains value 79 twice resulting in and indexing error when loading the event
        """
        print(f"Skipping problematic file: velo_event_{i}.json")
    else:    
        print(f"Loading file: velo_event_{i}.json")
        event_file = open(os.path.join("../DB/raw", f"velo_event_{i}.json"))
        json_data = json.loads(event_file.read())
        events.append(json_data) # type: ignore
        event_file.close()

In [ ]:
pipeline = TrackFollowingPipeline(events=events, intra_node=True) # type: ignore 

In [ ]:
Optimiser = Bayesian(learning_rate=0.1, max_iterations=243, target_score=0.001, objective="min") # type: ignore
optimal_parameters = pipeline.optimise_parameters(Optimiser, max_runs=2430) # DO NOT remove max_runs, chances are that this will run forever

In [ ]:
print(optimal_parameters) # Note these are just here for example...